In [1]:

import duckdb 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from src.feature_engineering import feature_engineering_lag , feature_engineering_delta
from src.random_forests import optim_hiperp , entrenamiento_rf,_imputacion
from src.red_dim import distanceMatrix , red_umap 

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb

c:\Users\chris\Data_science\Ms Data Mining UBA\2do_Cuatrimestre\EyF\competencia\bank-churn-prediction\env_comp1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'UMAP' from 'umap' (c:\Users\chris\Data_science\Ms Data Mining UBA\2do_Cuatrimestre\EyF\competencia\bank-churn-prediction\env_comp1\Lib\site-packages\umap\__init__.py)

In [2]:
data_path = "data/"
data_file = "competencia_01.csv"
df= pd.read_csv(data_path + data_file)

C:\Users\chris\AppData\Local\Temp\ipykernel_10708\4193464990.py:3: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(data_path + data_file)


#

In [3]:
cat_cols =[]
num_cols=[]
for c in df.columns:
    if (df[c].nunique() <= 5):
        cat_cols.append(c)
    else:
        num_cols.append(c)

In [4]:
lista_t=[c for c in list(map(lambda x : x if x[0]=='t' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_c=[c for c in list(map(lambda x : x if x[0]=='c' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_m=[c for c in list(map(lambda x : x if x[0]=='m' and x not in cat_cols else np.nan ,df.columns )) if pd.notna(c)]
lista_r=[c for c in df.columns if c not in (lista_t + lista_c + lista_m + cat_cols)]
print(len(lista_t) , len(lista_c) , len(lista_m) , len(lista_r) , len(cat_cols))
print(len(lista_t) + len(lista_c) + len(lista_m) +len(lista_r) + len(cat_cols))

0 48 45 42 20
155


In [5]:
col_drops=["cliente_edad","numero_de_cliente","cliente_antiguedad"]

In [6]:
cols_transf_previo=lista_m + lista_c+ lista_r
cols_trans = [c for c in cols_transf_previo if c not in col_drops]
cols_trans

['mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'mcuentas_saldo',
 'mautoservicio',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'mprestamos_personales',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'minversion1_pesos',
 'minversion1_dolares',
 'minversion2',
 'mpayroll',
 'mpayroll2',
 'mcuenta_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'mpagodeservicios',
 'mpagomiscuentas',
 'mcajeros_propios_descuentos',
 'mtarjeta_visa_descuentos',
 'mtarjeta_master_descuentos',
 'mcomisiones_mantenimiento',
 'mcomisiones_otras',
 'mforex_buy',
 'mforex_sell',
 'mtransferencias_recibidas',
 'mtransferencias_emitidas',
 'mextraccion_autoservicio',
 'mcheques_depositados',
 'mcheques_emitidos',
 'mcheque

In [7]:
df_2 = feature_engineering_lag(df ,cols_trans , 2)
df_3 = feature_engineering_delta(df_2 ,cols_trans , 2)

ejecucion lag finalizada
ejecucion delta finalizada


# OPTIMIZACION HIPERPARAMETROS


In [8]:
mes =202104
f = df_3["foto_mes"] == mes
df = df_3.loc[f]

X_train=df.drop(columns="clase_ternaria")
y_train = df["clase_ternaria"].copy()

In [14]:
n_trials=25
study=optim_hiperp(X_train,y_train,n_trials= n_trials)

[I 2025-09-14 15:04:59,880] Using an existing study with name 'exp_206_random-forest-opt' instead of creating a new one.


llegamos aca


[I 2025-09-14 15:07:07,284] Trial 10 finished with value: 343640000.0 and parameters: {'max_depth': 22, 'min_samples_split': 261, 'min_samples_leaf': 94, 'max_features': 0.2299885083191261}. Best is trial 10 with value: 343640000.0.
[I 2025-09-14 15:12:09,309] Trial 11 finished with value: 303960000.0 and parameters: {'max_depth': 20, 'min_samples_split': 1908, 'min_samples_leaf': 186, 'max_features': 0.5538808046995977}. Best is trial 10 with value: 343640000.0.
[I 2025-09-14 15:13:15,334] Trial 12 finished with value: 331420000.0 and parameters: {'max_depth': 11, 'min_samples_split': 43, 'min_samples_leaf': 99, 'max_features': 0.07087914882724963}. Best is trial 10 with value: 343640000.0.
[I 2025-09-14 15:14:04,424] Trial 13 finished with value: 326000000.0 and parameters: {'max_depth': 11, 'min_samples_split': 133, 'min_samples_leaf': 98, 'max_features': 0.07073947501806671}. Best is trial 10 with value: 343640000.0.
[I 2025-09-14 15:14:44,944] Trial 14 finished with value: 3267200

In [15]:
study.best_params

{'max_depth': 31,
 'min_samples_split': 235,
 'min_samples_leaf': 89,
 'max_features': 0.2688398569656583}

In [16]:
study.best_trial.params

{'max_depth': 31,
 'min_samples_split': 235,
 'min_samples_leaf': 89,
 'max_features': 0.2688398569656583}

In [ ]:
import json

best_params = study.best_trial.params

with open("src/outputs/random_forest/model/best_params.json", "w") as f:
    json.dump(best_params, f, indent=4)  # indent=4 para que quede legible

In [9]:
import json
with open("src/outputs/random_forest/model/best_params.json", "r") as f:
    best_params = json.load(f)
print(best_params)

{'max_depth': 31, 'min_samples_split': 235, 'min_samples_leaf': 89, 'max_features': 0.2688398569656583}


In [ ]:
# best_params = study.best_params
model = entrenamiento_rf(X_train, y_train,best_params)

In [ ]:
# import joblib
# model=joblib.load("src/outputs/random_forest/model/exp_206_random_forest_model_100.sav")

In [15]:
Xtrain_rf_imp=_imputacion(X_train)
Xtrain_rf_imp.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,delta_1_Visa_fechaalta,delta_2_Visa_fechaalta,delta_1_Visa_mconsumototal,delta_2_Visa_mconsumototal,delta_1_Visa_cconsumos,delta_2_Visa_cconsumos,delta_1_Visa_cadelantosefectivo,delta_2_Visa_cadelantosefectivo,delta_1_Visa_mpagominimo,delta_2_Visa_mpagominimo
3,249320580.0,202104.0,1.0,0.0,0.0,64.0,282.0,3122.15,19147.05,2472.63,...,30.0,61.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00
9,249368642.0,202104.0,1.0,0.0,0.0,52.0,259.0,-3278.95,-955.47,3558.26,...,30.0,61.0,2094.49,-6045.89,2.0,0.0,0.0,0.0,1888.53,1431.06
15,249483501.0,202104.0,1.0,0.0,0.0,52.0,319.0,713.47,11305.03,300.13,...,30.0,61.0,899.46,-4.29,1.0,-1.0,0.0,0.0,527.85,551.31
21,249493416.0,202104.0,1.0,0.0,0.0,56.0,259.0,4113.23,34716.24,3647.86,...,30.0,61.0,87.97,-2274.16,0.0,0.0,0.0,0.0,0.00,-117.30
27,249494219.0,202104.0,1.0,0.0,0.0,57.0,376.0,-1902.65,-111.00,1068.38,...,30.0,61.0,-1083.00,2187.42,2.0,-1.0,0.0,0.0,-2862.12,-2334.27


# REDUCCION DE DIMENSIONALIDAD

In [16]:
md = distanceMatrix(model, Xtrain_rf_imp)

NameError: name 'distanceMatrix' is not defined